In [1]:
DOWNLOAD = False
if DOWNLOAD:
  !pip install ultralytics --quiet
  !yolo settings wandb=True --quiet
from pathlib import Path
import os
from ultralytics import YOLO
import kagglehub
import wandb
from google.colab import userdata
from ultralytics import settings

api_key = userdata.get('Wandb')
wandb.login(key = api_key)
settings.update({"wandb": True})

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: egedalgic to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
dataset_path = kagglehub.dataset_download("lakshaytyagi01/fruit-detection")

Using Colab cache for faster access to the 'fruit-detection' dataset.


# Loading Dataset

In [3]:
def find_split_root(root: Path):
    candidates = []
    for p, dirs, files in os.walk(root):
        p = Path(p)
        if (p / "train" / "images").is_dir() and (p / "valid" / "images").is_dir():
            candidates.append(p)
    candidates = sorted(candidates, key=lambda x: ("fruit" not in x.as_posix().lower(), len(x.as_posix())))
    return candidates[0] if candidates else None

split_root = find_split_root(dataset_path)

In [ ]:
train_images = split_root / "train" / "images"
val_images   = split_root / "valid" / "images"
test_images  = split_root / "test" / "images"

names = [
    "Apple", "Banana", "Grape", "Orange", "Pineapple", "Watermelon"
]

data_yaml_path = Path("/content/data.yaml") if Path("/content").exists() else Path.cwd() / "data.yaml"

data_yaml_text = f"""
# Auto-generated for Ultralytics
path: {split_root.as_posix()}
train: {train_images.as_posix()}
val: {val_images.as_posix()}
{'test: ' + test_images.as_posix() if test_images.is_dir() else ''}

names: {names}
nc: {len(names)}
""".strip()

with open(data_yaml_path, "w", encoding="utf-8") as f:
    f.write(data_yaml_text)
print(f"[Info] Wrote data.yaml to: {data_yaml_path}")

[Info] Wrote data.yaml to: /content/data.yaml


# Model Training

In [ ]:
TRAIN = False

if TRAIN:
  model = YOLO("yolo11n.pt")
  results = model.train(
    data=str(data_yaml_path),
    epochs = 5,
    imgsz=640, #resoultion bigger the better but slower
    batch=32,#batch size = -1 for automatic batch size selection
    iou=0.6, #mape was calculated with iou=0.6
    conf=0.001, #confidence threshold for object detection (True if above this value)
    device=0, #use GPU
    project="YOLO",
    name="coco_finetune",
    save_json=True,
    #cache = True, #cache images for faster training; cache = "disk" to cache to disk (slower but better than nothing and more useful for large datasets)
    #amp = True, #automatic mixed precision (faster training with less memory usage)
    #optimizer = "auto", #select best optimizer automatically 
    #patience=... #early stopping
    #also one can add data augmentation parameters here
    )



# Notes

* All the metrics, run logs and visualizations are stored in wandb. Only save the best weights manually or export the model all together.